# Build a language model for a specific subject


One of the state-of-the art solutions to build NLP application is using word embeddings to
compute similarities between texts. Generally, they are vector representations of words that
are capable of capturing the context of a word in a document or relation with other words.

Word2vec is a two-layer neural network that processes text. Its input is a text corpus and its output is a set of vectors: feature vectors for words in that corpus. The purpose and usefulness of Word2vec is to group the vectors of similar words together in vectorspace. This way. it is able to detect similarities mathematically [1].

A pre-trained set of vectors containing part of Google News dataset can be downloaded and used [2].
In case of technical fields, however, in order to achieve a better accuracy, a new language model should be trained,
in order to obtain the specific vocabulary that may be missing from the news dataset.

## Build up an input corpus

The first step is to build a corpus of specialized words, that can be used to get the vocabulary.
In order to do that, we decided to use papers stored in Zenodo [3], since it offers a free end point for
querying. In order to do that, we raise multiple queries to the server and we store the results locally.
The approach that we selected was to download the pdf files and transform them to txt, using a command lines tool
for Linux. To speed up the process and to avoid duplicates, we save the files in the '/tmp' folder and check, each time, if the file is already there. In case it is, we load the text, which is already pre-processed. I case of builing a language model, it is better to not have duplicates, since it can change the values attached to words.

The next step is to cleanup the text. In order to to that, the first step is to remove all punctation and replace non-alpha characters to spaces. Them all the words are transformed to lowercase and lemmatized to their basis word. Then,
all the stop words are removed, together with the non-English and short words. In this case, it is important to keep the  abbreviations, technical words which are not the English dictionary and short words related to the concerned field.
Them all the abreviations are replaced by the corresponding words sequence, to be able to compute similarities right. The clean-up texts are then saved in the corresponding text files in the '/tmp' folder.

## Build a language model

The corpus of documents is directly sent to word2vec library of gensim [4] and then saved in a binary file for further
use.


[1] https://skymind.ai/wiki/word2vec
[2] https://code.google.com/archive/p/word2vec/
[3] https://zenodo.org/
[4] https://radimrehurek.com/gensim/models/word2vec.html

In [16]:
%run "NLP_clustering.ipynb"
%run "Utils_Zenodo.ipynb"

def get_entries_from_zenodo(query):
    
    total_articles = get_zenodo_entries(query)
    documents = []
    
    for article in total_articles:
        doi = article['doi']
        mod_doi = doi.replace('/', '-')
        mod_doi = mod_doi.replace('.', '-')
        mod_doi = mod_doi.replace(':', '-')

        #look in tmp folder if there is a file containing the doi
        # if there is, just read the file and move to the next entry
        if os.path.isfile('E:\\' + doi + ".txt"):
            with open('E:\\' + doi + ".txt", encoding="utf8") as f:
                doc_list = f.read().split(' ')
        else:
            print(article['files'][0]['links']['self'])
            # download the file and preprocess in the information
            doc_list = save_pdf_and_get_text(article['files'][0]['links']['self'])
            # overwrite the doi temporary file
            with open('E:\\' + mod_doi + ".txt", 'w', encoding="utf8") as f:
                f.write(' '.join(doc_list))
                f.close()
            # add the text to the corpus   
            if doc_list:
                documents.append(doc_list)
 
    return documents


biologic_process_list_of_words = ['cell adhesion', 'signal transduction', 'extracellular stimulus', 'hormone secretion',
            'intracellular signaling cascade', 'organ development', 'cell differentiation', 'blood vessel', 'biogenesis',
            'cellular physiological process', 'organization biogenesis', 'biologic process']
    
if __name__ == "__main__":
    #documents is a list of lists
    documents = get_entries_from_zenodo(biologic_process_list_of_words[0])
    for i in range(1, len(biologic_process_list_of_words)):
        documents.extend(get_entries_from_zenodo(biologic_process_list_of_words[i]))
    
    model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
    model.train(documents,total_examples=len(documents),epochs=10)
    model.wv.save_word2vec_format(MODEL_TO_TRAIN, binary=True)
    

https://zenodo.org/api/files/3363b9d0-5b2f-4434-bf8c-ed5547bba9ea/article.pdf
https://zenodo.org/api/files/ec7313e8-07a8-4a91-ae60-a683ee081852/1.5000588.pdf
https://zenodo.org/api/files/26289b95-0ba2-490a-ba32-509714ad5238/Improving_the_seeding_effectiveness_of_stromal_and_epithelial_cell_cultures_102-107_final.pdf
https://zenodo.org/api/files/426611f2-5d47-43e8-921e-898354cb4cb3/11-Messias%20A_MR_2013_OPEN_Lithograph-Moulded%20Poly-L-co-D%2CL%20Lactide%20Porous%20Membranes%20for%20Osteoblastic%20CultureLithograph-Moulded%20Poly-L-co-D%2CL%20Lactide%20Porous%20Membranes%20for%20Osteoblastic%20Culture.pdf
https://zenodo.org/api/files/40f10072-c3a8-4f9d-9291-161d7b742c01/pdf.pdf
https://zenodo.org/api/files/baedac70-a477-43e1-9d6c-a4ff91c6c05e/ACP-2-110.pdf
https://zenodo.org/api/files/78f7b8ff-6ab0-4ace-a4d3-6d4bd2a9fb25/5629.pdf
https://zenodo.org/api/files/49d97a2a-97e6-4f6a-810d-a917f5e9b2af/article.pdf
https://zenodo.org/api/files/fe448e83-1041-43c8-909b-206a9ff860a6/6057.pdf
https

https://zenodo.org/api/files/a89dda82-0842-4a1e-8b65-cfb2d0030340/document_bH8nJuo.pdf
https://zenodo.org/api/files/3bd4f063-4a04-4cbe-96f9-e1e4f709e650/14682.pdf
https://zenodo.org/api/files/9f559fd5-764f-4b12-bab3-6ab6882fc4a4/3.pdf
https://zenodo.org/api/files/d3629cee-2c18-4f22-8755-a4639eb76d8a/10009239.pdf
https://zenodo.org/api/files/7eadc218-84f6-435e-8284-d9953a8b2beb/article.pdf
https://zenodo.org/api/files/52b4751e-9093-462d-bf20-b33eadd11c4e/DJORDJEVIC_BS_41-2_2019.pdf
https://zenodo.org/api/files/4f85b719-488a-4d79-bbc9-138c2e9ed875/article.pdf
https://zenodo.org/api/files/2ec9a769-a185-4751-bce6-f767193557a7/135.Mo%20Ali-Role%20of%20Microbiome%20and%20STT%20in%20Combating%20Melanoma.pdf
https://zenodo.org/api/files/1c225421-b9ca-4a62-85b3-a3b4b34db59d/article.pdf
https://zenodo.org/api/files/f5e37329-72ce-4e0c-af70-4d0c592d50e3/article.pdf
https://zenodo.org/api/files/39d2fc20-38b9-4805-831f-26c29ee3d5a9/150924_Templin_Basel_MipTec.pdf
https://zenodo.org/api/files/ce9f4db

https://zenodo.org/api/files/c034da38-06fe-4454-a63a-6cba24b76e8a/article.pdf
https://zenodo.org/api/files/d2b4e567-b975-4de1-b9ea-e753d5d3105a/12964_2018_Article_233.pdf
https://zenodo.org/api/files/4e46e38d-2629-44ec-b9a4-e257346cb5fe/BLSW_YUAN.pdf
https://zenodo.org/api/files/36dc9d4e-b89b-4d35-9ad8-22c87c80cdf5/article.pdf
https://zenodo.org/api/files/d4c7e337-a229-4b72-bda4-57e8ca76eac9/source.pdf
https://zenodo.org/api/files/c61c785c-98e0-4780-92a1-ece072c2aafd/Supplementary%20Figures.pdf
https://zenodo.org/api/files/2d6db9dc-1038-43e3-b2af-7f24f7c2108a/article.pdf
https://zenodo.org/api/files/ad2fa953-1140-4998-bdc0-bfb843284ad6/Walzer2014.pdf
https://zenodo.org/api/files/f202a4f8-fd67-4520-bfb8-2fc864f63180/9781329899735_frontcover_content_backcover.pdf
https://zenodo.org/api/files/7ebe294c-71c3-42c7-b5b5-8165a5f2676f/9781329900523_frontcover_content_backcover.pdf
https://zenodo.org/api/files/e6bcc43d-cb00-4916-8115-41a65cc83268/69.pdf
https://zenodo.org/api/files/648b179e-dff1

https://zenodo.org/api/files/730bae5c-abb7-4d79-bf7f-8619ae71b9a5/Differential_expression_of_canonical_and_non-canonical_Wnt_ligands_in_ameloblastoma.pdf
https://zenodo.org/api/files/d1966bed-6c4e-41c4-88ce-578bb1cf2bc5/BioDiscovery_S.Gopalakrishna_Poster_Presenter.pdf
https://zenodo.org/api/files/fcfa26b0-3aa4-4319-bc64-23c9fc06d476/article.pdf
https://zenodo.org/api/files/985b8cb8-79ce-4e39-8e3b-17bb0bec5678/1362055076-Lee332012BJMMR2426.pdf
https://zenodo.org/api/files/e368beac-ef42-45d8-ae21-a31082279d1a/5.%20IJZAB%20ID%20No.%20334.pdf
https://zenodo.org/api/files/98002623-8462-46c2-9297-1729b11a7307/%2860-69%29Growth%20factors%20in%20platelet%20rich%20plasma-format.pdf
https://zenodo.org/api/files/c034da38-06fe-4454-a63a-6cba24b76e8a/article.pdf
https://zenodo.org/api/files/dbcf1721-4cca-4343-8ac8-3a75f6dc1214/12859_2019_Article_2816.pdf
https://zenodo.org/api/files/f97948e7-312f-4636-8ff2-fd74ab155be4/Bioprinting_Basel_Life_Science_Week_final_2.pdf
https://zenodo.org/api/files/c7

https://zenodo.org/api/files/1f3eaf9e-7c48-4462-82d0-2f42bb0480ff/9999482.pdf
https://zenodo.org/api/files/8baf6d85-70fa-418a-a79a-8fe4fef166ef/Preprint-Ko.pdf
https://zenodo.org/api/files/5e93a9ba-ca29-4104-95d7-dd64486bf381/JPR-8-4-173-185.PDF
https://zenodo.org/api/files/bd1f8d53-dae7-4618-b442-b568c707b4f0/13.pdf
https://zenodo.org/api/files/eb14d069-70a1-4f72-87c8-b647b0928bd5/Proof%20Acepted%20plus%20License%20statment.pdf
https://zenodo.org/api/files/b3c2d229-41ef-4462-9ec6-532118ddea2c/Thesis_20-10-14.pdf
https://zenodo.org/api/files/bf233310-6a2d-4f9d-bd32-ea114dad37fc/7.pdf
https://zenodo.org/api/files/5303b831-e5ab-43f3-85c2-183035bedebe/233.For%202%20%282%29-converted.pdf
https://zenodo.org/api/files/9e3a9c8f-8a79-4709-9a15-bdddf301d285/10005306.pdf
https://zenodo.org/api/files/04e55c2b-3d60-402d-8078-7180d1dfbfd5/ejh-62-1-2867.pdf
https://zenodo.org/api/files/bffd6c39-1b57-4137-a843-afedcb1300ca/article.pdf
https://zenodo.org/api/files/23398982-5a14-4eca-873f-b41f358b2a33/

https://zenodo.org/api/files/9f9f7ee2-2800-4ab9-93ca-875569f6e1bf/9781329900561_frontcover_content_backcover.pdf


In [17]:
print(len(model.wv.vocab))
#print(documents)

#print(model.wv.most_similar('solar'))
#print(model.wv.most_similar('photovoltaic'))

print(model.wv.most_similar('genetic'))
print(model.wv.most_similar('chromosome'))
print(model.wv.most_similar('cell'))




11273
[('development', 0.9135873913764954), ('prognostic', 0.9120426177978516), ('network', 0.9042519330978394), ('architecture', 0.8985012769699097), ('impact', 0.8921080827713013), ('behavior', 0.8849215507507324), ('environmental', 0.8732221126556396), ('pathogenesis', 0.8717740774154663), ('therapeutic', 0.8692610263824463), ('clinical', 0.8619660139083862)]
[('homology', 0.9691707491874695), ('cognate', 0.9635659456253052), ('milieu', 0.9367101788520813), ('structurally', 0.9331323504447937), ('translocate', 0.9324544668197632), ('reestablish', 0.9312657117843628), ('posse', 0.9283502101898193), ('obscure', 0.9280160665512085), ('stalk', 0.9250900745391846), ('catalytic', 0.9247335195541382)]
[('tissue', 0.9093469381332397), ('human', 0.8972060680389404), ('vessel', 0.8961011171340942), ('cellular', 0.8946257829666138), ('protein', 0.8868533372879028), ('unraveled', 0.8854694366455078), ('expression', 0.8852810859680176), ('gene', 0.8829666376113892), ('molecular', 0.8777455091476

In [7]:
print(len(model.wv.vocab))
print(documents)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

